In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import csv

In [32]:
# Create a new instance of the Chrome driver
service = Service()
options = webdriver.ChromeOptions()
options.add_argument(r'--user-data-dir=C:\Users\bethl\AppData\Local\Google\Chrome\User Data')
driver = webdriver.Chrome(service=service, options=options)

# Define the URL of the job listings page
url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=%22+data%20scientist+%22&sc.keyword=%22+data%20scientist+%22&locT=&locId=&jobType="
driver.get(url)

# Allow time for the page to load
time.sleep(5)

# Function to scroll to the bottom and click 'Show more jobs' button
def load_more_jobs():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    try:
        show_more_button = driver.find_element(By.XPATH, "//button[@data-test='load-more']")
        ActionChains(driver).move_to_element(show_more_button).click().perform()
        print("Clicked 'Show more jobs' button.")
        return True
    except Exception as e:
        print("Could not find or click 'Show more jobs' button:", e)
        return False

# Scroll and click 'Show more jobs' button several times
for _ in range(0):  # Adjust the range as needed to click more times
    if not load_more_jobs():
        break
    time.sleep(5)  # Wait for new jobs to load

# Find all job title elements with the class 'JobCard_jobTitle___7I6y'
job_titles = driver.find_elements(By.CLASS_NAME, 'JobCard_jobTitle___7I6y')

# List to hold job details
job_details = []

# Initialize the CSV file and write the header
with open('glassdoor_jobs.csv', 'w', newline='', encoding='utf-8') as output_file:
    keys = ["Job Title", "Salary Estimate", "Job Description", "Rating", "Company Name", "Location", 
            "Size", "Founded", "Type of ownership", "Industry", "Sector", "Revenue"]
    dict_writer = csv.DictWriter(output_file, fieldnames=keys)
    dict_writer.writeheader()

    # Loop through the job titles, print their text and URLs, open each URL in a new tab, then close the tab
    for job in job_titles:
        job_title = job.text.strip()
        job_url = job.get_attribute('href')
        
        # Open each job listing in a new tab
        driver.execute_script(f"window.open('{job_url}', '_blank');")
        time.sleep(2)  # Adjust the sleep time as needed
        
        # Switch to the new tab
        driver.switch_to.window(driver.window_handles[1])
        
        # Extract job details
        try:
            salary_estimate = driver.find_element(By.XPATH, "//div[@class='SalaryEstimate_salaryRange__brHFy']").text
            salary_estimate = salary_estimate.replace("â€“", "–")
        except:
            salary_estimate = "N/A"
            
        try:
            job_description = driver.find_element(By.XPATH, "//div[@class, 'JobDetails_jobDescription__uW_fK']")
            job_description = job_description_div.get_text(separator='\n', strip=True)
                                                                           
        except:
            job_description = "N/A"
        
        try:
            rating = driver.find_element(By.XPATH, "//div[@class='EmployerProfile_ratingContainer__ul0Ef']//span").text
        except:
            rating = "N/A"
        
        try:
            company_name = driver.find_element(By.XPATH, "//h4[@class='heading_Heading__BqX5J heading_Subhead__Ip1aW']").text
        except:
            company_name = "N/A"
        
        try:
            location = driver.find_element(By.XPATH, "//div[@class='JobDetails_location__mSg5h']").text
        except:
            location = "N/A"
        
        
        try:
            size = driver.find_element(By.XPATH, "//div[span[text()='Size']]//following-sibling::div").text
        except:
            size = "N/A"
        
        try:
            founded = driver.find_element(By.XPATH, "//div[span[text()='Founded']]//following-sibling::div").text
        except:
            founded = "N/A"
        
        try:
            type_of_ownership = driver.find_element(By.XPATH, "//div[span[text()='Type']]//following-sibling::div").text
        except:
            type_of_ownership = "N/A"
        
        try:
            industry = driver.find_element(By.XPATH, "//div[span[text()='Industry']]//following-sibling::div").text
        except:
            industry = "N/A"
        
        try:
            sector = driver.find_element(By.XPATH, "//div[span[text()='Sector']]//following-sibling::div").text
        except:
            sector = "N/A"
        
        try:
            revenue = driver.find_element(By.XPATH, "//div[span[text()='Revenue']]//following-sibling::div").text
        except:
            revenue = "N/A"
        
        job_detail = {
            "Job Title": job_title,
            "Salary Estimate": salary_estimate,
            "Job Description": job_description,
            "Rating": rating,
            "Company Name": company_name,
            "Location": location,
            "Size": size,
            "Founded": founded,
            "Type of ownership": type_of_ownership,
            "Industry": industry,
            "Sector": sector,
            "Revenue": revenue,
        }

        job_details.append(job_detail)
        
        # Write the current job detail to the CSV file
        dict_writer.writerow(job_detail)
        
        # Close the new tab
        driver.close()
        
        # Switch back to the original tab
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)  # Adjust the sleep time as needed

# Close the browser
driver.quit()

print("Job details saved to glassdoor_jobs.csv")


Job details saved to glassdoor_jobs.csv
